In [1]:
from autogen import ConversableAgent
from openai import OpenAI
from pathlib import Path

import LLM_configs
import Prompt_dics
from LLM_configs import *
from Agents import *
from tools import *

In [2]:
# 加载 .env 文件
load_dotenv()

True

## 文件提取

In [15]:
text = file_extraction("test files/前海知行7月实地.m4a-文稿-转写结果.docx")

In [4]:
PPT = file_extraction("test files/上海象上投资.pptx")

## Agent设计

In [16]:
quant_dict = {
    'quant_idea':[],
    'factor_composition':[],
    'factor_explanation':[],
    'model_explanation':[],
    'factor_update':[],
    'strategy_execution':[]
}

In [17]:
market_neutral = fill_placeholders(['中证500指增', '股指期货','0%'], Prompt_dics.Other_prompt_dic['中性多头/对冲端'])

In [19]:
market_neutral = ''

In [20]:
def report_writing(weak_config, strong_config,prompts,component_dict, requirements, updating):
    extraction_agent = extraction_assistant(weak_config)
    
    integrate_agent = integrate_assistant(strong_config,requirements)
    
    for prompt, component in zip(prompts, list(component_dict.keys())):
        reply = extraction_agent.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": prompt, "role": "user"}])
        component_dict[component].append(reply)
    
    component_dict['hedging tools'] = market_neutral
    
    final_report = integrate_agent.generate_reply(messages=[{"content": str(component_dict), "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整，长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    non_updating_report = final_report
    
    if updating:
        old_report_relevant_part = input("请把上一篇报告的配置逻辑部分复制到此处")
        final_report = integrate_agent.generate_reply(messages=[{"content": "旧报告: "+old_report_relevant_part+"新报告:"+str(final_report), "role": "system"},
                                                                {"content": "你是一个报告审查和撰写专员，现在你有一篇旧报告和一篇新报告，你要做的是在旧报告的基础上，判断新报告中的增量信息，并将新报告中的增量信息与旧报告中的现有信息相结合，完成一篇更新的报告，如果两个报告中对某一个地方有冲突，则以新报告为准。", "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整.长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    
    cost_list = [extraction_agent.get_total_usage(),integrate_agent.get_total_usage()]
    
    return final_report,non_updating_report, component_dict, cost_list

In [21]:
Ali_report,first_Ali_report, Quant_component_dic, cost_list = report_writing(LLM_configs.Qwen_weak_llm_config, LLM_configs.Qwen_strong_llm_config, Prompt_dics.Strategy_prompt_dic['Simple Quant Strategy'],quant_dict, Prompt_dics.Strategy_requirements_dic['Simple Quant Strategy'],True)


>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...


In [22]:
print(Ali_report)

管理人的量化策略整体框架基于强逻辑性和高可解释性的原则，侧重于人工精选因子与线性模型的应用，旨在确保策略的透明度与可控性。策略核心围绕着约2000个量价因子构建，经过严格筛选，实际投入使用的因子数量约为300个，其中包括250个隔日因子、200个实盘因子以及50个日内因子，所有因子均为量价类型。尽管策略中未直接实盘运行基本面因子和另类因子，但策略设计时已考虑这些因子对策略收益的潜在贡献，并在风险管理环节融入了如市盈率PE等基本面因子，以增强策略的全面性与稳定性。

因子挖掘与筛选过程高度依赖人工智慧，通过深入市场逻辑分析，确保因子的可持续性和逻辑合理性，相较于自动化机器学习方法，人工挖掘更注重因子的长期稳定性和可解释性。在因子处理与合成方面，管理人采用线性模型作为主要框架，如排序打分和简单线性回归，这些模型不仅易于理解与调整，而且在因子表现异动时能够迅速识别并应对。尽管新报告未详尽阐述新因子的入库标准，但从策略执行的严谨性来看，管理人显然有一套完善的因子评估与纳入机制，包括基于IC、t-value值、相关性等指标的绩效计算，每周五例行进行。

在具体执行层面，管理人对股票池进行了细致的处理，初始剔除了市值极端（大于1000亿或小于30亿）、股价过低、PB过高、PE为负以及日成交额不足3000万的股票，随后根据指数行业权重构建股池，每月调整。这一过程确保了投资组合的市场适应性和流动性。值得注意的是，策略在2023年底经历了一次重要迭代，针对模型低配高波动股票的问题，新增了波动率因子至风控模型中，以期与指数波动率保持一致，从而捕捉更多高波动股票的投资机会。迭代后的回测数据显示，虽然短期内（如2023年7月至11月）策略表现得以提升，追上了市场同类私募，但长期年化收益率预计会有所下降，大约每年减少1%。

风险管理方面，管理人严格控制行业偏离度在申万一级行业分类标准下的2%以内，同时确保Barra风格因子暴露不超过0.5个标准差，有效降低了非系统性风险。持股数量方面，500指增策略调整为持有500-600只股票，而1000指增策略的持股数量信息未更新，但旧报告中提到700-800只。年化双边换手率调整为更加保守的15%左右，显著低于旧报告中的60-100倍，显示了管理人在交易成本控制上的优化。

关于对冲策略，知行稳健1号采取市场中性策略，500指增与1000指增分别占3

In [23]:
print(first_Ali_report)

管理人的量化策略整体倾向于强逻辑和高可解释性，避免过度依赖机器学习的暴力挖掘，注重因子的市场逻辑和可解释性。策略核心围绕人工精选因子，偏好线性模型以确保策略的透明度和可控性。策略中包含约2000个量价类因子，实际生产使用的约为300个，其中量价因子占据主导，同时融入基本面因子（如市盈率PE）进行风险管理，虽未详述另类因子，但显示策略考虑了多维度市场影响因素。因子挖掘主要依靠人工，强调因子的持续性和逻辑性，相比自动化的批量生成，人工挖掘的因子展现出更好的持续性。模型构建上，管理人偏好使用线性模型，如排序打分和简单回归，这些方法便于理解和调整，特别是在因子表现反转时能快速定位问题。尽管未明确新因子入库的具体条件，但策略执行层面显示了严谨的操作流程，包括使用自研及考虑引入外部下单算法，持股数量由过去的700-800只调整至500-600只以强化风险控制，年化双边换手率维持在15%左右，行业偏离度控制在2%以内，风格偏离度严格管理，如贝塔偏离度不超过0.5，且对Barra风格因子实施严格控制。风险管理还包括剔除低市值股票及利用PE比率等因子进行风险调节。策略调整灵活，依据市场动态调整持股数量、市值门槛，并适时加入新的风险控制因子。整个策略执行过程中，管理人展现了对模型优化的重视，通过因子归因分析和定期回溯检验来确保策略的有效性。至于对冲方法，当前提供的信息未包含具体对冲工具或策略的使用情况。


In [24]:
cost_list

[{'total_cost': 0.37469199999999997,
  'qwen-plus': {'cost': 0.37469199999999997,
   'prompt_tokens': 89275,
   'completion_tokens': 1466,
   'total_tokens': 90741}},
 {'total_cost': 0.27296,
  'qwen-max': {'cost': 0.27296,
   'prompt_tokens': 3503,
   'completion_tokens': 1107,
   'total_tokens': 4610}}]

In [25]:
Quant_component_dic

{'quant_idea': ['管理人的量化策略整体上偏向于强逻辑和可解释性，而不是依赖暴力挖掘或大量使用机器学习方法。策略制定过程中注重因子的逻辑性和可解释性，这意味着在构建模型时，更侧重于理解市场行为背后的逻辑，并确保所使用的因子能够被清晰解释，而非仅仅依赖于数据挖掘得到的结果。\n\n这种方法论体现在以下几个方面：\n1. **因子挖掘**：采用人工挖掘为主的方式，强调因子的持续性和市场逻辑，而非大规模自动化挖掘。\n2. **模型构建**：偏好线性模型，这类模型更容易理解和调整，尤其是在因子反转等异常情况出现时，能够更好地分析和解决问题。\n3. **风险管理**：在风险管理方面，采取严格的控制措施，如市值、行业偏差等方面的限制，以减少潜在风险。\n4. **策略调整**：策略调整基于市场变化和风险因子的动态评估，而非简单的模型优化或参数调整。\n\n整体而言，该策略倾向于稳健和可控的方法，重视策略的逻辑清晰性和风险控制的有效性。'],
 'factor_composition': ['管理人的策略中因子的具体情况如下：\n\n- 因子总数：量价类因子大约有2000个。\n- 生产中的因子数量：约300个。\n- 因子构成：\n  - 量价类因子占主导地位。\n  - 基本面因子也被纳入考量，尤其是在风险管理方面，例如对PE（市盈率）的控制。\n  - 另类因子的使用情况未详细说明，但从对话中可以看出，管理人团队不仅依赖传统的量价和技术分析，还考虑了基本面因素和其他可能影响市场的新因素。\n\n总体而言，量价类因子构成了因子库的主要部分，而基本面和其他类型的因子则作为补充，用于风险管理和策略优化。'],
 'factor_explanation': ['根据提供的信息，管理人提到了几种不同的因子类型及其来源，具体如下：\n\n- **量价因子**：这部分因子主要基于量价数据进行挖掘。管理人提到量价因子随着时间的推移变得越来越难以挖掘有效因子，时效性较快。\n- **基本面因子**：例如 PE（市盈率）因子，管理人在 5 月份之后将其加入到了优化器中作为一种风险控制手段。此外，还提到了现金流因子，虽然在过去可能不是一个很强的风险控制因子，但在特定情况下可能会发挥作用。\n- **订单流因子**：这类因子在成交量下降的过程中表现较差。\n- **技术指标**：

In [ ]:
def report_updating(weak_config, strong_config,prompts,component_dict, requirements):
    